In [ ]:
 !pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=4da47573f231f27892d31efa67a8c8e1026be44e8445e2b506fa61e5b5f3da16
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MySparkApp').enableHiveSupport().getOrCreate()

In [ ]:
from pyspark.sql import functions as F

In [ ]:
data = [("101","Henry", 25), ("102","Jack", 30), ("103","Peter", 35)]
columns = ["ID", "Name", "Age"]
df = spark.createDataFrame(data, columns)
df.show()
print(type(df))

+---+-----+---+
| ID| Name|Age|
+---+-----+---+
|101|Henry| 25|
|102| Jack| 30|
|103|Peter| 35|
+---+-----+---+

<class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
df.createOrReplaceTempView("stud_view")
result = spark.sql("SELECT * FROM stud_view")
result.show()
result = spark.sql("SELECT * FROM stud_view WHERE age>25")
result.show()

+---+-----+---+
| ID| Name|Age|
+---+-----+---+
|101|Henry| 25|
|102| Jack| 30|
|103|Peter| 35|
+---+-----+---+

+---+-----+---+
| ID| Name|Age|
+---+-----+---+
|102| Jack| 30|
|103|Peter| 35|
+---+-----+---+



In [ ]:
df.write.saveAsTable("stud_table")
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)



In [ ]:
spark.sql("CREATE TABLE IF NOT EXISTS new_stud_table AS SELECT * FROM stud_table")

DataFrame[]

In [ ]:
spark.sql("DESCRIBE new_stud_table").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|      ID|   string|   NULL|
|    Name|   string|   NULL|
|     Age|   bigint|   NULL|
+--------+---------+-------+



In [ ]:
spark.sql("SHOW COLUMNS FROM new_stud_table").show()

+--------+
|col_name|
+--------+
|      ID|
|    Name|
|     Age|
+--------+



In [ ]:
spark.sql("ALTER TABLE new_stud_table ADD COLUMN  branch STRING")

DataFrame[]

In [ ]:
spark.sql("DESCRIBE new_stud_table").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|      ID|   string|   NULL|
|    Name|   string|   NULL|
|     Age|   bigint|   NULL|
|  branch|   string|   NULL|
+--------+---------+-------+



In [ ]:
spark.sql("INSERT INTO TABLE new_stud_table VALUES ('104','David',26,'AIDS'),('105','Sara',27,'AIDS')")

DataFrame[]

In [ ]:
spark.sql("SELECT * FROM new_stud_table order by age").show()

+---+-----+---+------+
| ID| Name|Age|branch|
+---+-----+---+------+
|101|Henry| 25|  NULL|
|104|David| 26|  AIDS|
|105| Sara| 27|  AIDS|
|102| Jack| 30|  NULL|
|103|Peter| 35|  NULL|
+---+-----+---+------+



In [ ]:
df = spark.read.table("new_stud_table")
df.show()

+---+-----+---+------+
| ID| Name|Age|branch|
+---+-----+---+------+
|104|David| 26|  AIDS|
|102| Jack| 30|  NULL|
|103|Peter| 35|  NULL|
|105| Sara| 27|  AIDS|
|101|Henry| 25|  NULL|
+---+-----+---+------+



In [ ]:
from pyspark.sql.functions import expr
updated_df = df.withColumn("age_plus_5",expr("age + 5"))
updated_df.show()

+---+-----+---+------+----------+
| ID| Name|Age|branch|age_plus_5|
+---+-----+---+------+----------+
|104|David| 26|  AIDS|        31|
|102| Jack| 30|  NULL|        35|
|103|Peter| 35|  NULL|        40|
|105| Sara| 27|  AIDS|        32|
|101|Henry| 25|  NULL|        30|
+---+-----+---+------+----------+



In [ ]:
updated_df = df.withColumn("age",expr("age + 1"))
updated_df.show()

+---+-----+---+------+
| ID| Name|age|branch|
+---+-----+---+------+
|104|David| 27|  AIDS|
|102| Jack| 31|  NULL|
|103|Peter| 36|  NULL|
|105| Sara| 28|  AIDS|
|101|Henry| 26|  NULL|
+---+-----+---+------+



In [ ]:
from pyspark.sql.functions import when
updated_df = df.withColumn("is_adult",when(expr("age>=18"),"Yes").otherwise("No"))
updated_df.show()

+---+-----+---+------+--------+
| ID| Name|Age|branch|is_adult|
+---+-----+---+------+--------+
|104|David| 26|  AIDS|     Yes|
|102| Jack| 30|  NULL|     Yes|
|103|Peter| 35|  NULL|     Yes|
|105| Sara| 27|  AIDS|     Yes|
|101|Henry| 25|  NULL|     Yes|
+---+-----+---+------+--------+



In [ ]:
from pyspark.sql.functions import col
updated_df = df.withColumn("is_adult",when(col("age")>=18,"Yes").otherwise("No"))
updated_df.show()

+---+-----+---+------+--------+
| ID| Name|Age|branch|is_adult|
+---+-----+---+------+--------+
|104|David| 26|  AIDS|     Yes|
|102| Jack| 30|  NULL|     Yes|
|103|Peter| 35|  NULL|     Yes|
|105| Sara| 27|  AIDS|     Yes|
|101|Henry| 25|  NULL|     Yes|
+---+-----+---+------+--------+



In [ ]:
spark.sql("ALTER TABLE new_stud_table ADD COLUMN  Mark_1 INT")
spark.sql("ALTER TABLE new_stud_table ADD COLUMN  Mark_2 INT")
spark.sql("ALTER TABLE new_stud_table ADD COLUMN  Mark_3 INT")

DataFrame[]

In [ ]:
spark.sql("DESCRIBE new_stud_table").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|      ID|   string|   NULL|
|    Name|   string|   NULL|
|     Age|   bigint|   NULL|
|  branch|   string|   NULL|
|  Mark_1|      int|   NULL|
|  Mark_2|      int|   NULL|
|  Mark_3|      int|   NULL|
+--------+---------+-------+



In [ ]:
# Load the table 'new_stud_table' into a DataFrame.
df = spark.sql("SELECT * FROM new_stud_table")

# Now you can use the withColumn and when functions to add the 'is_mark' column.
from pyspark.sql.functions import col
updated_df = df.withColumn("is_mark",when(col("Mark_1")>=50,"Fail").otherwise("Pass"))
updated_df.show()

+---+-----+---+------+------+------+------+-------+
| ID| Name|Age|branch|Mark_1|Mark_2|Mark_3|is_mark|
+---+-----+---+------+------+------+------+-------+
|104|David| 26|  AIDS|  NULL|  NULL|  NULL|   Pass|
|102| Jack| 30|  NULL|  NULL|  NULL|  NULL|   Pass|
|103|Peter| 35|  NULL|  NULL|  NULL|  NULL|   Pass|
|105| Sara| 27|  AIDS|  NULL|  NULL|  NULL|   Pass|
|101|Henry| 25|  NULL|  NULL|  NULL|  NULL|   Pass|
+---+-----+---+------+------+------+------+-------+

